In [21]:
from pyspark.sql import SparkSession, functions as F
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
import pandas as pd

# Ініціалізація SparkSession
spark = SparkSession.builder.appName("Process Cleaned JSON Data").getOrCreate()

df = spark.read.json("/home/yevhen/Проекти/butsa/json/filtered_players.json")

print(df.count())

filtered_players = (
    df.filter(
        (df.primaryPosition == "Cf")  #|
        # (df.primaryPosition == "Cm") |
        # (df.primaryPosition == "Lm") |
        # (df.primaryPosition == "Rm")  |
        # (df.primaryPosition == "Am")
    )
    .sort(df.rTalent.desc(), df.power.desc())
    .select(
        "id",
        "name",
        "country_name",
        "primaryPosition",
        "secondaryPosition",
        "club",
        "rTalent",
        "power",
        "team_id",
    )
)

# Виводимо результати
filtered_players.show(300, truncate=False)

# Перетворюємо Spark DataFrame у Pandas DataFrame
pandas_df = filtered_players.toPandas()

# Зберігаємо у файл Excel
output_path = "/home/yevhen/Проекти/butsa/json/filtered_players.xlsx"
# with pd.ExcelWriter(output_path, engine="openpyxl", mode="a") as writer:
#     pandas_df.to_excel(writer, sheet_name="Forwards", index=False)

wb = load_workbook(output_path)
ws = wb["Forwards"]  # Відкриваємо потрібний лист

# Форматування клітинок: заповнення кольором, якщо rTalent > 4.5
fill_highlight = PatternFill(start_color="FF7F7F", end_color="FF7F7F", fill_type="solid") # ADD8E6 - light blue, FF7F7F - light red

for row in ws.iter_rows(min_row=2, max_row=ws.max_row):  # Починаємо з другого рядка
    club_cell = row[6 - 1]  # Індекс колонки (мінус 1, бо індекси Python починаються з 0)
    if club_cell.value == "Ницца": # Ницца, Виллем II
        for cell in row:
            cell.fill = fill_highlight

# Зберігаємо файл
wb.save(output_path)

282
+------+---------------------+-------------------+---------------+-----------------+---------------------+-------+-----+---------------------------------+
|id    |name                 |country_name       |primaryPosition|secondaryPosition|club                 |rTalent|power|team_id                          |
+------+---------------------+-------------------+---------------+-----------------+---------------------+-------+-----+---------------------------------+
|851766|Альваро Мората       |Испания            |Cf             |                 |Вильярреал           |5.1    |22.45|https://www.butsa.ru/roster/305  |
|853108|Фидель Боливар       |Венесуэла          |Cf             |Am               |Кильмес              |5.0    |37.92|https://www.butsa.ru/roster/10743|
|851955|Басилио Рубиньо      |Боливия            |Cf             |Am               |Кильмес              |5.0    |32.69|https://www.butsa.ru/roster/10743|
|852992|Зорепад Калапач      |Украина            |Cf             |

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Ініціалізація SparkSession
spark = SparkSession.builder.appName("Average rTalent by Club").getOrCreate()

# Завантаження JSON у Spark DataFrame
df = spark.read.json("/home/yevhen/Проекти/butsa/json/filtered_players.json")

# Обчислення середнього rTalent по Club
avg_rTalent_by_club = (
    df.groupBy("club")  # Групуємо за країною та клубом
    .agg(
        F.avg("rTalent").alias("avg_rTalent"),  # Обчислюємо середнє значення rTalent
        F.avg("power").alias("avg_power"),  # Обчислюємо середнє значення power
        F.count("id").alias("count"),  # Рахуємо кількість гравців у клубі
    )
    .filter(F.col("count") >= 6)
    .select("club", "avg_rTalent", "avg_power", "count")
    .orderBy(F.desc("avg_rTalent"))  # Сортуємо за спаданням середнього rTalent
)

# Виводимо результати
avg_rTalent_by_club.show(40, truncate=False)

25/04/16 11:50:09 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.0.117 instead (on interface wlo1)
25/04/16 11:50:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/16 11:50:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/16 11:50:26 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+--------------+-----------------+------------------+-----+
|club          |avg_rTalent      |avg_power         |count|
+--------------+-----------------+------------------+-----+
|Кильмес       |4.966666666666667|30.693333333333335|9    |
|Факел         |4.814285714285715|20.434285714285714|7    |
|Вильярреал    |4.762499999999999|21.767500000000002|8    |
|Солфорд       |4.688888888888889|22.236666666666665|9    |
|Спартак Москва|4.51             |30.052            |10   |
|Аль Барк Тарим|4.5              |29.61333333333333 |12   |
|Уракан Манати |4.463636363636363|27.15727272727273 |11   |
|Газ Метан     |4.454545454545454|34.78363636363636 |11   |
|Виллем II     |4.441666666666667|22.7375           |12   |
|ПФК Рось      |4.300000000000001|26.27             |10   |
|АнтА          |4.288888888888889|23.74888888888889 |9    |
|Ницца         |4.233333333333333|19.689999999999994|12   |
|Х.Б.С.        |4.166666666666667|29.135            |6    |
+--------------+-----------------+------

In [22]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive


def upload_excel_to_google_drive(
    file_path,
    file_name,
    mime_type="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
):
    """
    Завантажує Excel-файл у Google Диск як Google Sheet.
    Якщо файл із такою назвою вже існує, він буде перезаписаний.

    :param file_path: Шлях до локального файлу Excel.
    :param file_name: Назва файлу, який буде створено або перезаписано на Google Диску.
    :param mime_type: Тип файлу для завантаження (за замовчуванням Google Sheet).
    """
    # Авторизація через GoogleAuth
    gauth = GoogleAuth()
    gauth.LoadClientConfigFile(
        "/home/yevhen/Проекти/butsa/json/client_secret_935743862394-45jpsgc3uv5g890ivtfj5843hj6p4u65.apps.googleusercontent.com.json"
    )
    gauth.LocalWebserverAuth()  # Відкриває браузер для авторизації
    drive = GoogleDrive(gauth)

    # Пошук файлу з такою ж назвою
    file_list = drive.ListFile(
        {"q": f"title='{file_name}' and trashed=false"}
    ).GetList()

    if file_list:
        # Якщо файл знайдено, оновлюємо його
        file = file_list[0]  # Беремо перший файл із такою назвою
        file.SetContentFile(file_path)
        file.Upload()
        print(f"Файл '{file_name}' успішно оновлено на Google Диску.")
    else:
        # Якщо файл не знайдено, створюємо новий
        file = drive.CreateFile({"title": file_name, "mimeType": mime_type})
        file.SetContentFile(file_path)
        file.Upload()


upload_excel_to_google_drive(
    file_path="/home/yevhen/Проекти/butsa/json/filtered_players.xlsx",
    file_name="Filtered players",
)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=935743862394-45jpsgc3uv5g890ivtfj5843hj6p4u65.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.
Файл 'Filtered players' успішно оновлено на Google Диску.


In [1]:
import json
from collections import defaultdict
import os

def calculate_division_strength(file_paths):
    # Словник для зберігання сумарної сили та кількості гравців у кожному дивізіоні
    division_stats = defaultdict(lambda: {"total_power": 0, "player_count": 0})

    for file_path in file_paths:
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)

        # Отримуємо назву країни з ключа "name"
        country_name = data.get("name", "Unknown Country")

        # Проходимо по всіх командах у файлі
        for team_id, team_data in data.get("teams", {}).items():
            division_type = team_data.get("divisiontype", "Unknown Division")
            division_key = f"{country_name} - Division {division_type}"

            players = team_data.get("players", {})

            # Проходимо по всіх гравцях команди
            for player_id, player_data in players.items():
                power = player_data.get("power")
                if power is not None:
                    division_stats[division_key]["total_power"] += power
                    division_stats[division_key]["player_count"] += 1

    # Обчислення середньої сили для кожного дивізіону
    division_avg_power = {
        division_key: stats["total_power"] / stats["player_count"]
        for division_key, stats in division_stats.items()
        if stats["player_count"] > 0
    }

    return division_avg_power

# Список файлів з країнами
country_files_dir = "/home/yevhen/Проекти/butsa/json/cleaned_responses/"
country_files = [os.path.join(country_files_dir, file) for file in os.listdir(country_files_dir) if file.endswith(".txt")]

# Обчислення середньої сили для всіх файлів
division_strengths = calculate_division_strength(country_files)

# Виведення результатів
for division_key, avg_power in division_strengths.items():
    print(f"{division_key}: Average Power: {avg_power:.2f}")

Литва - Division 1: Average Power: 135.05
Литва - Division 2: Average Power: 102.83
Йемен - Division 2: Average Power: 96.73
Йемен - Division 1: Average Power: 110.48
Индия - Division 1: Average Power: 113.02
Албания - Division 2: Average Power: 109.74
Албания - Division 1: Average Power: 112.76
Монако - Division 1: Average Power: 107.87
Северная Ирландия - Division 1: Average Power: 127.96
Северная Ирландия - Division 2: Average Power: 99.47
Бангладеш - Division 1: Average Power: 127.62
Азербайджан - Division 1: Average Power: 117.32
Азербайджан - Division 2: Average Power: 98.62
Лихтенштейн - Division 1: Average Power: 133.22
Лихтенштейн - Division 2: Average Power: 108.97
Гонконг - Division 1: Average Power: 121.24
Гвинея-Бисау - Division 1: Average Power: 117.22
Франция - Division 3: Average Power: 99.54
Франция - Division 1: Average Power: 111.75
Франция - Division 2: Average Power: 122.02
Франция - Division 6: Average Power: 90.45
Франция - Division 4: Average Power: 105.12
Франц

In [1]:
import json
import os

def calculate_average_power_all_divisions(file_paths):
    # Словник для зберігання сумарної сили та кількості гравців у всіх дивізіонах
    total_power = 0
    total_players = 0

    for file_path in file_paths:
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)

        # Проходимо по всіх командах у файлі
        for team_id, team_data in data.get("teams", {}).items():
            players = team_data.get("players", {})

            # Проходимо по всіх гравцях команди
            for player_id, player_data in players.items():
                power = player_data.get("power")
                if power is not None:
                    total_power += power
                    total_players += 1

    # Обчислення середньої сили для всіх дивізіонів
    if total_players > 0:
        average_power = total_power / total_players
    else:
        average_power = 0

    return average_power

# Список файлів з країнами
country_files_dir = "/home/yevhen/Проекти/butsa/json/cleaned_responses/"
country_files = [os.path.join(country_files_dir, file) for file in os.listdir(country_files_dir) if file.endswith(".txt")]

# Обчислення середньої сили для всіх дивізіонів
average_power_all_divisions = calculate_average_power_all_divisions(country_files)

# Виведення результату
print(f"Середня сила всіх дивізіонів: {average_power_all_divisions:.2f}")

Середня сила всіх дивізіонів: 109.95
